## Perfect.csv用のLightGBM

In [ ]:
def train_lgbm(params, t_list, column, fold, df):
    print(f'fold:{fold}')
    print(f'columns:{column}')
    
    deberta_df = pd.read_csv(data_path + f'fold{fold}.csv')
    df['cohesion'] = deberta_df['cohesion']
    df['syntax'] = deberta_df['syntax']
    df['vocabulary'] = deberta_df['vocabulary']
    df['phraseology'] = deberta_df['phraseology']
    df['grammar'] = deberta_df['grammar']
    df['conventions'] = deberta_df['conventions']
    df['fold'] = deberta_df['fold']
    
    df_train = df[df.fold != fold]
    df_valid = df[df.fold == fold]
        
    # 学習データ
    x_train = df_train.drop(t_list, axis = 1)
    y_train = df_train[f'target-{column}']
        
    # 評価用データ
    x_valid = df_valid.drop(t_list, axis = 1)
    y_valid = df_valid[f'target-{column}']
        
    categorical_features = []
        
    #LightGBM用の訓練データセットを作成
    lgb_train = lgbm.Dataset(x_train,y_train,categorical_feature=categorical_features,free_raw_data=False)

    #LightGBM用の評価データセットを作成
    lgb_eval = lgbm.Dataset(x_valid,y_valid,reference=lgb_train,categorical_feature=categorical_features,free_raw_data=False)

    #LightGBMの学習
    lgbm_model = lgbm.train(params,
                            lgb_train,
                            valid_sets=[lgb_train,lgb_eval],
                            verbose_eval=100,
                            # 計算回数
                            num_boost_round=3000,
                            early_stopping_rounds= 200
                           )
    
    preds = lgbm_model.predict(x_valid)
    rmse = np.sqrt(mean_squared_error(y_valid, preds))
    print(rmse)
    
    file = f'lgbm_{column}_{fold}.pkl'
    pickle.dump(lgbm_model, open(save_path + file, 'wb'))
    
    del df, lgb_train, lgb_eval
    return rmse, lgbm_model

In [ ]:
0.44905
0.42415
0.430138
0.413216

In [ ]:
0.447626
0.423544
0.430148
0.415338

In [ ]:
(0.444326 + 0.419902 + 0.424559 + 0.40874)/4